### Preprocessing

In [3]:
import numpy as np

from scipy.io import loadmat

import pandas as pd

import datetime as date

from dateutil.relativedelta import relativedelta



cols = ['age', 'gender', 'path', 'face_score1', 'face_score2']


wiki_mat = 'wiki/wiki.mat'


wiki_data = loadmat(wiki_mat)



del wiki_mat


wiki = wiki_data['wiki']



wiki_photo_taken = wiki[0][0][1][0]

wiki_full_path = wiki[0][0][2][0]

wiki_gender = wiki[0][0][3][0]

wiki_face_score1 = wiki[0][0][6][0]

wiki_face_score2 = wiki[0][0][7][0]



wiki_path = []



for path in wiki_full_path:

    wiki_path.append('wiki_crop/' + path[0])


wiki_genders = []




for n in range(len(wiki_gender)):

    if wiki_gender[n] == 1:

        wiki_genders.append('male')

    else:

        wiki_genders.append('female')



wiki_dob = []



for file in wiki_path:

    wiki_dob.append(file.split('_')[2])



wiki_age = []



for i in range(len(wiki_dob)):

    try:

        d1 = date.datetime.strptime(wiki_dob[i][0:10], '%Y-%m-%d')

        d2 = date.datetime.strptime(str(wiki_photo_taken[i]), '%Y')

        rdelta = relativedelta(d2, d1)

        diff = rdelta.years

    except Exception as ex:

        print(ex)

        diff = -1

    wiki_age.append(diff)



final_wiki = np.vstack((wiki_age, wiki_genders, wiki_path, wiki_face_score1, wiki_face_score2)).T


final_wiki_df = pd.DataFrame(final_wiki)


final_wiki_df.columns = cols

meta = pd.DataFrame(final_wiki_df)

#meta = pd.concat((final_imdb_df, final_wiki_df))



meta = meta[meta['face_score1'] != '-inf']

meta = meta[meta['face_score2'] == 'nan']



meta = meta.drop(['face_score1', 'face_score2'], axis=1)



meta = meta.sample(frac=1)



meta.to_csv('meta.csv', index=False)

time data '829-09-08' does not match format '%Y-%m-%d'
time data '1823-12-00' does not match format '%Y-%m-%d'
time data '1953-10-00' does not match format '%Y-%m-%d'
time data '1984-15-03' does not match format '%Y-%m-%d'
time data '1963-02-00' does not match format '%Y-%m-%d'
time data '1977-04-00' does not match format '%Y-%m-%d'
time data '1946-05-00' does not match format '%Y-%m-%d'
time data '647-05-15' does not match format '%Y-%m-%d'
time data '11-03-00' does not match format '%Y-%m-%d'
time data '1980-05-00' does not match format '%Y-%m-%d'
time data '1960-02-00' does not match format '%Y-%m-%d'
time data '1975-03-00' does not match format '%Y-%m-%d'
time data '1952-11-00' does not match format '%Y-%m-%d'
time data '1974-04-00' does not match format '%Y-%m-%d'
time data '1996-25-05' does not match format '%Y-%m-%d'
time data '1914-02-00' does not match format '%Y-%m-%d'


### Dataset Generation

In [4]:
import pandas as pd

import numpy as np

import cv2

import os

from sklearn.model_selection import train_test_split



# Loading the data
b

meta = pd.read_csv('meta.csv')



# Deleting the age column as we dont need it

meta = meta.drop(['age'], axis=1)



# Spliting the dataset into train and test set

D_train, D_test = train_test_split(meta, test_size=0.1, random_state=42)



# The dataset contains more male faces that female faces. This can couse some problems.

# One feature can start dominating on other feature. To solve this I am selecting equal number of male and female faces in the training set

D_train_male = D_train[D_train['gender'] == 'male']

D_train_female = D_train[D_train['gender'] == 'female']



no_male = len(D_train_male)

no_female = len(D_train_female)



if no_male > no_female:

    extra = D_train_male[no_female:]

    D_train_male = D_train_male[0:no_female]



    D_test = pd.concat((D_test, extra))

else:

    extra = D_train_male[no_male:]

    D_train_male = D_train_male[0:no_male]



    D_test = pd.concat((D_test, extra))



D_train = pd.concat((D_train_male, D_train_female))



# Shuffling the dataset

D_train = D_train.sample(frac=1)

D_test = D_test.sample(frac=1)



# Generating folder struture for the data

output_dir_train_male = 'dataset/gender/train/male'

output_dir_train_female = 'dataset/gender/train/female'



if not os.path.exists(output_dir_train_male):

    os.makedirs(output_dir_train_male)



if not os.path.exists(output_dir_train_female):

    os.makedirs(output_dir_train_female)



output_dir_test_male = 'dataset/gender/test/male'

output_dir_test_female = 'dataset/gender/test/female'



if not os.path.exists(output_dir_test_male):

    os.makedirs(output_dir_test_male)



if not os.path.exists(output_dir_test_female):

    os.makedirs(output_dir_test_female)



# Finally processing the image training and testing set

male_tr_ctr = 0
female_tr_ctr = 0

male_test_ctr = 0
female_test_ctr = 0


counter_1 = 0

for image in D_train.values:

#     img = cv2.imread(image[1], 1)

#     img = cv2.resize(img, (128,128))
    
    try:
        img = cv2.imread(image[1], 1)
        img = cv2.resize(img, (128,128))

    except Exception as e:
        print(str(e))

    if image[0] == 'male':

        cv2.imwrite('dataset/gender/train/male/' + str(male_tr_ctr) + '.jpg', img)
        male_tr_ctr +=1
        
    else:

        cv2.imwrite('dataset/gender/train/female/' + str(female_tr_ctr) + '.jpg', img)
        female_tr_ctr +=1
    
    counter_1 += 1

    print('--('+str(counter_1)+')Processing--')

   
counter_2 = 0

for image in D_test.values:

#     img = cv2.imread(image[1], 1)

#     img = cv2.resize(img, (128,128))
    
    try:
        
        img = cv2.imread(image[1], 1)
        img = cv2.resize(img, (128,128))

    except Exception as e:
        
        print(str(e))
    

    if image[0] == 'male':

        cv2.imwrite('dataset/gender/test/male/' + str(male_test_ctr) + '.jpg', img)
        male_test_ctr +=1

    else:

        cv2.imwrite('dataset/gender/test/female/' + str(female_test_ctr) + '.jpg', img)
        female_test_ctr +=1

    counter_2 +=1
        
    print('--('+str(counter_2)+')Processing--')

--(1)Processing--
--(2)Processing--
--(3)Processing--
--(4)Processing--
--(5)Processing--
--(6)Processing--
--(7)Processing--
--(8)Processing--
--(9)Processing--
--(10)Processing--
--(11)Processing--
--(12)Processing--
--(13)Processing--
--(14)Processing--
--(15)Processing--
--(16)Processing--
--(17)Processing--
--(18)Processing--
--(19)Processing--
--(20)Processing--
--(21)Processing--
--(22)Processing--
--(23)Processing--
--(24)Processing--
--(25)Processing--
--(26)Processing--
--(27)Processing--
--(28)Processing--
--(29)Processing--
--(30)Processing--
--(31)Processing--
--(32)Processing--
--(33)Processing--
--(34)Processing--
--(35)Processing--
--(36)Processing--
--(37)Processing--
--(38)Processing--
--(39)Processing--
--(40)Processing--
--(41)Processing--
--(42)Processing--
--(43)Processing--
--(44)Processing--
--(45)Processing--
--(46)Processing--
--(47)Processing--
--(48)Processing--
--(49)Processing--
--(50)Processing--
--(51)Processing--
--(52)Processing--
--(53)Processing--
--

--(419)Processing--
--(420)Processing--
--(421)Processing--
--(422)Processing--
--(423)Processing--
--(424)Processing--
--(425)Processing--
--(426)Processing--
--(427)Processing--
--(428)Processing--
--(429)Processing--
--(430)Processing--
--(431)Processing--
--(432)Processing--
--(433)Processing--
--(434)Processing--
--(435)Processing--
--(436)Processing--
--(437)Processing--
--(438)Processing--
--(439)Processing--
--(440)Processing--
--(441)Processing--
--(442)Processing--
--(443)Processing--
--(444)Processing--
--(445)Processing--
--(446)Processing--
--(447)Processing--
--(448)Processing--
--(449)Processing--
--(450)Processing--
--(451)Processing--
--(452)Processing--
--(453)Processing--
--(454)Processing--
--(455)Processing--
--(456)Processing--
--(457)Processing--
--(458)Processing--
--(459)Processing--
--(460)Processing--
--(461)Processing--
--(462)Processing--
--(463)Processing--
--(464)Processing--
--(465)Processing--
--(466)Processing--
--(467)Processing--
--(468)Processing--


--(839)Processing--
--(840)Processing--
--(841)Processing--
--(842)Processing--
--(843)Processing--
--(844)Processing--
--(845)Processing--
--(846)Processing--
--(847)Processing--
--(848)Processing--
--(849)Processing--
--(850)Processing--
--(851)Processing--
--(852)Processing--
--(853)Processing--
--(854)Processing--
--(855)Processing--
--(856)Processing--
--(857)Processing--
--(858)Processing--
--(859)Processing--
--(860)Processing--
--(861)Processing--
--(862)Processing--
--(863)Processing--
--(864)Processing--
--(865)Processing--
--(866)Processing--
--(867)Processing--
--(868)Processing--
--(869)Processing--
--(870)Processing--
--(871)Processing--
--(872)Processing--
--(873)Processing--
--(874)Processing--
--(875)Processing--
--(876)Processing--
--(877)Processing--
--(878)Processing--
--(879)Processing--
--(880)Processing--
--(881)Processing--
--(882)Processing--
--(883)Processing--
--(884)Processing--
--(885)Processing--
--(886)Processing--
--(887)Processing--
--(888)Processing--


--(1238)Processing--
--(1239)Processing--
--(1240)Processing--
--(1241)Processing--
--(1242)Processing--
--(1243)Processing--
--(1244)Processing--
--(1245)Processing--
--(1246)Processing--
--(1247)Processing--
--(1248)Processing--
--(1249)Processing--
--(1250)Processing--
--(1251)Processing--
--(1252)Processing--
--(1253)Processing--
--(1254)Processing--
--(1255)Processing--
--(1256)Processing--
--(1257)Processing--
--(1258)Processing--
--(1259)Processing--
--(1260)Processing--
--(1261)Processing--
--(1262)Processing--
--(1263)Processing--
--(1264)Processing--
--(1265)Processing--
--(1266)Processing--
--(1267)Processing--
--(1268)Processing--
OpenCV(3.4.2) C:\Miniconda3\conda-bld\opencv-suite_1534379934306\work\modules\imgproc\src\resize.cpp:4044: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

--(1269)Processing--
--(1270)Processing--
--(1271)Processing--
--(1272)Processing--
--(1273)Processing--
--(1274)Processing--
--(1275)Processing--
--(1276)Processing--
--

--(2010)Processing--
--(2011)Processing--
--(2012)Processing--
--(2013)Processing--
--(2014)Processing--
--(2015)Processing--
--(2016)Processing--
--(2017)Processing--
--(2018)Processing--
--(2019)Processing--
--(2020)Processing--
--(2021)Processing--
--(2022)Processing--
--(2023)Processing--
--(2024)Processing--
--(2025)Processing--
--(2026)Processing--
--(2027)Processing--
--(2028)Processing--
--(2029)Processing--
--(2030)Processing--
--(2031)Processing--
--(2032)Processing--
--(2033)Processing--
--(2034)Processing--
--(2035)Processing--
--(2036)Processing--
--(2037)Processing--
--(2038)Processing--
--(2039)Processing--
--(2040)Processing--
--(2041)Processing--
--(2042)Processing--
--(2043)Processing--
--(2044)Processing--
--(2045)Processing--
--(2046)Processing--
--(2047)Processing--
--(2048)Processing--
--(2049)Processing--
--(2050)Processing--
--(2051)Processing--
--(2052)Processing--
--(2053)Processing--
--(2054)Processing--
--(2055)Processing--
--(2056)Processing--
--(2057)Proce

...

--(19699)Processing--
--(19700)Processing--
--(19701)Processing--
--(19702)Processing--
--(19703)Processing--
--(19704)Processing--
--(19705)Processing--
--(19706)Processing--
--(19707)Processing--
--(19708)Processing--
--(19709)Processing--
--(19710)Processing--
--(19711)Processing--
--(19712)Processing--
--(19713)Processing--
--(19714)Processing--
--(19715)Processing--
--(19716)Processing--
--(19717)Processing--
--(19718)Processing--
--(19719)Processing--
--(19720)Processing--
--(19721)Processing--
--(19722)Processing--
--(19723)Processing--
--(19724)Processing--
--(19725)Processing--
--(19726)Processing--
--(19727)Processing--
--(19728)Processing--
--(19729)Processing--
--(19730)Processing--
--(19731)Processing--
--(19732)Processing--
--(19733)Processing--
--(19734)Processing--
--(19735)Processing--
--(19736)Processing--
--(19737)Processing--
--(19738)Processing--
--(19739)Processing--
--(19740)Processing--
--(19741)Processing--
--(19742)Processing--
--(19743)Processing--
--(19744)P

--(20076)Processing--
--(20077)Processing--
--(20078)Processing--
--(20079)Processing--
--(20080)Processing--
--(20081)Processing--
--(20082)Processing--
--(20083)Processing--
--(20084)Processing--
--(20085)Processing--
--(20086)Processing--
--(20087)Processing--
--(20088)Processing--
--(20089)Processing--
--(20090)Processing--
--(20091)Processing--
--(20092)Processing--
--(20093)Processing--
--(20094)Processing--
--(20095)Processing--
--(20096)Processing--
--(20097)Processing--
--(20098)Processing--
--(20099)Processing--
--(20100)Processing--
--(20101)Processing--
--(20102)Processing--
--(20103)Processing--
--(20104)Processing--
--(20105)Processing--
--(20106)Processing--
--(20107)Processing--
--(20108)Processing--
--(20109)Processing--
--(20110)Processing--
--(20111)Processing--
--(20112)Processing--
--(20113)Processing--
--(20114)Processing--
--(20115)Processing--
--(20116)Processing--
--(20117)Processing--
--(20118)Processing--
--(20119)Processing--
--(20120)Processing--
--(20121)P

--(20454)Processing--
--(20455)Processing--
--(20456)Processing--
--(20457)Processing--
--(20458)Processing--
--(20459)Processing--
--(20460)Processing--
--(20461)Processing--
--(20462)Processing--
--(20463)Processing--
--(20464)Processing--
--(20465)Processing--
--(20466)Processing--
--(20467)Processing--
--(20468)Processing--
--(20469)Processing--
--(20470)Processing--
--(20471)Processing--
--(20472)Processing--
--(20473)Processing--
--(20474)Processing--
--(20475)Processing--
--(20476)Processing--
--(20477)Processing--
--(20478)Processing--
--(20479)Processing--
--(20480)Processing--
--(20481)Processing--
--(20482)Processing--
--(20483)Processing--
--(20484)Processing--
--(20485)Processing--
--(20486)Processing--
--(20487)Processing--
--(20488)Processing--
--(20489)Processing--
--(20490)Processing--
--(20491)Processing--
--(20492)Processing--
--(20493)Processing--
--(20494)Processing--
--(20495)Processing--
--(20496)Processing--
--(20497)Processing--
--(20498)Processing--
--(20499)P

--(20829)Processing--
--(20830)Processing--
--(20831)Processing--
--(20832)Processing--
--(20833)Processing--
--(20834)Processing--
--(20835)Processing--
--(20836)Processing--
--(20837)Processing--
--(20838)Processing--
--(20839)Processing--
--(20840)Processing--
--(20841)Processing--
--(20842)Processing--
--(20843)Processing--
--(20844)Processing--
--(20845)Processing--
--(20846)Processing--
--(20847)Processing--
--(20848)Processing--
--(20849)Processing--
--(20850)Processing--
--(20851)Processing--
--(20852)Processing--
--(20853)Processing--
--(20854)Processing--
--(20855)Processing--
--(20856)Processing--
--(20857)Processing--
--(20858)Processing--
--(20859)Processing--
--(20860)Processing--
--(20861)Processing--
--(20862)Processing--
--(20863)Processing--
--(20864)Processing--
--(20865)Processing--
--(20866)Processing--
--(20867)Processing--
--(20868)Processing--
--(20869)Processing--
--(20870)Processing--
--(20871)Processing--
--(20872)Processing--
--(20873)Processing--
--(20874)P

--(21207)Processing--
--(21208)Processing--
--(21209)Processing--
--(21210)Processing--
--(21211)Processing--
--(21212)Processing--
--(21213)Processing--
--(21214)Processing--
--(21215)Processing--
--(21216)Processing--
--(21217)Processing--
--(21218)Processing--
--(21219)Processing--
--(21220)Processing--
--(21221)Processing--
--(21222)Processing--
--(21223)Processing--
--(21224)Processing--
--(21225)Processing--
--(21226)Processing--
--(21227)Processing--
--(21228)Processing--
--(21229)Processing--
--(21230)Processing--
--(21231)Processing--
--(21232)Processing--
--(21233)Processing--
--(21234)Processing--
--(21235)Processing--
--(21236)Processing--
--(21237)Processing--
--(21238)Processing--
--(21239)Processing--
--(21240)Processing--
--(21241)Processing--
--(21242)Processing--
--(21243)Processing--
--(21244)Processing--
--(21245)Processing--
--(21246)Processing--
--(21247)Processing--
--(21248)Processing--
--(21249)Processing--
--(21250)Processing--
--(21251)Processing--
--(21252)P

### Custom SFD Face Detection

BBOX

In [9]:
from __future__ import print_function

import os,sys,cv2,random,datetime,time,math

import argparse

import numpy as np

import torch



try:

    from iou import IOU

except:

    # IOU cython speedup 10x

    def IOU(ax1,ay1,ax2,ay2,bx1,by1,bx2,by2):

        sa = abs((ax2-ax1)*(ay2-ay1))

        sb = abs((bx2-bx1)*(by2-by1))

        x1,y1 = max(ax1,bx1),max(ay1,by1)

        x2,y2 = min(ax2,bx2),min(ay2,by2)

        w  = x2 - x1

        h  = y2 - y1

        if w<0 or h<0: return 0.0

        else: return 1.0*w*h/(sa+sb-w*h)



def bboxlog(x1,y1,x2,y2,axc,ayc,aww,ahh):

    xc,yc,ww,hh = (x2+x1)/2,(y2+y1)/2,x2-x1,y2-y1

    dx,dy = (xc-axc)/aww,(yc-ayc)/ahh

    dw,dh = math.log(ww/aww),math.log(hh/ahh)

    return dx,dy,dw,dh



def bboxloginv(dx,dy,dw,dh,axc,ayc,aww,ahh):

    xc,yc = dx*aww+axc, dy*ahh+ayc

    ww,hh = math.exp(dw)*aww,math.exp(dh)*ahh

    x1,x2,y1,y2 = xc-ww/2,xc+ww/2,yc-hh/2,yc+hh/2

    return x1,y1,x2,y2



def nms(dets, thresh):

    if 0==len(dets): return []

    x1,y1,x2,y2,scores = dets[:, 0],dets[:, 1],dets[:, 2],dets[:, 3],dets[:, 4]

    areas = (x2 - x1 + 1) * (y2 - y1 + 1)

    order = scores.argsort()[::-1]



    keep = []

    while order.size > 0:

        i = order[0]

        keep.append(i)

        xx1,yy1 = np.maximum(x1[i], x1[order[1:]]),np.maximum(y1[i], y1[order[1:]])

        xx2,yy2 = np.minimum(x2[i], x2[order[1:]]),np.minimum(y2[i], y2[order[1:]])



        w,h = np.maximum(0.0, xx2 - xx1 + 1),np.maximum(0.0, yy2 - yy1 + 1)

        ovr = w*h / (areas[i] + areas[order[1:]] - w*h)



        inds = np.where(ovr <= thresh)[0]

        order = order[inds + 1]



    return keep



def encode(matched, priors, variances):

    """Encode the variances from the priorbox layers into the ground truth boxes

    we have matched (based on jaccard overlap) with the prior boxes.

    Args:

        matched: (tensor) Coords of ground truth for each prior in point-form

            Shape: [num_priors, 4].

        priors: (tensor) Prior boxes in center-offset form

            Shape: [num_priors,4].

        variances: (list[float]) Variances of priorboxes

    Return:

        encoded boxes (tensor), Shape: [num_priors, 4]

    """



    # dist b/t match center and prior's center

    g_cxcy = (matched[:, :2] + matched[:, 2:])/2 - priors[:, :2]

    # encode variance

    g_cxcy /= (variances[0] * priors[:, 2:])

    # match wh / prior wh

    g_wh = (matched[:, 2:] - matched[:, :2]) / priors[:, 2:]

    g_wh = torch.log(g_wh) / variances[1]

    # return target for smooth_l1_loss

    return torch.cat([g_cxcy, g_wh], 1)  # [num_priors,4]



def decode(loc, priors, variances):

    """Decode locations from predictions using priors to undo

    the encoding we did for offset regression at train time.

    Args:

        loc (tensor): location predictions for loc layers,

            Shape: [num_priors,4]

        priors (tensor): Prior boxes in center-offset form.

            Shape: [num_priors,4].

        variances: (list[float]) Variances of priorboxes

    Return:

        decoded bounding box predictions

    """



    boxes = torch.cat((

        priors[:, :2] + loc[:, :2] * variances[0] * priors[:, 2:],

        priors[:, 2:] * torch.exp(loc[:, 2:] * variances[1])), 1)

    boxes[:, :2] -= boxes[:, 2:] / 2

    boxes[:, 2:] += boxes[:, :2]

    return boxes

SFD Model

In [10]:
import torch

import torch.nn as nn

from torch.autograd import Variable

import torch.nn.functional as F



class L2Norm(nn.Module):

    def __init__(self,n_channels, scale=1.0):

        super(L2Norm,self).__init__()

        self.n_channels = n_channels

        self.scale = scale

        self.eps = 1e-10

        self.weight = nn.Parameter(torch.Tensor(self.n_channels))

        self.weight.data *= 0.0

        self.weight.data += self.scale



    def forward(self, x):

        norm = x.pow(2).sum(dim=1, keepdim=True).sqrt()+self.eps

        x = x / norm * self.weight.view(1,-1,1,1)

        return x



class s3fd(nn.Module):

    def __init__(self):

        super(s3fd, self).__init__()

        self.conv1_1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)

        self.conv1_2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)



        self.conv2_1 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)

        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)



        self.conv3_1 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)

        self.conv3_2 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)

        self.conv3_3 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)



        self.conv4_1 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)

        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)

        self.conv4_3 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)



        self.conv5_1 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)

        self.conv5_2 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)

        self.conv5_3 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)



        self.fc6     = nn.Conv2d(512, 1024, kernel_size=3, stride=1, padding=3)

        self.fc7     = nn.Conv2d(1024, 1024, kernel_size=1, stride=1, padding=0)



        self.conv6_1 = nn.Conv2d(1024, 256, kernel_size=1, stride=1, padding=0)

        self.conv6_2 = nn.Conv2d(256, 512, kernel_size=3, stride=2, padding=1)



        self.conv7_1 = nn.Conv2d(512, 128, kernel_size=1, stride=1, padding=0)

        self.conv7_2 = nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1)



        self.conv3_3_norm = L2Norm(256,scale=10)

        self.conv4_3_norm = L2Norm(512,scale=8)

        self.conv5_3_norm = L2Norm(512,scale=5)



        self.conv3_3_norm_mbox_conf = nn.Conv2d(256, 4, kernel_size=3, stride=1, padding=1)

        self.conv3_3_norm_mbox_loc  = nn.Conv2d(256, 4, kernel_size=3, stride=1, padding=1)

        self.conv4_3_norm_mbox_conf = nn.Conv2d(512, 2, kernel_size=3, stride=1, padding=1)

        self.conv4_3_norm_mbox_loc  = nn.Conv2d(512, 4, kernel_size=3, stride=1, padding=1)

        self.conv5_3_norm_mbox_conf = nn.Conv2d(512, 2, kernel_size=3, stride=1, padding=1)

        self.conv5_3_norm_mbox_loc  = nn.Conv2d(512, 4, kernel_size=3, stride=1, padding=1)



        self.fc7_mbox_conf     = nn.Conv2d(1024, 2, kernel_size=3, stride=1, padding=1)

        self.fc7_mbox_loc      = nn.Conv2d(1024, 4, kernel_size=3, stride=1, padding=1)

        self.conv6_2_mbox_conf = nn.Conv2d(512, 2, kernel_size=3, stride=1, padding=1)

        self.conv6_2_mbox_loc  = nn.Conv2d(512, 4, kernel_size=3, stride=1, padding=1)

        self.conv7_2_mbox_conf = nn.Conv2d(256, 2, kernel_size=3, stride=1, padding=1)

        self.conv7_2_mbox_loc  = nn.Conv2d(256, 4, kernel_size=3, stride=1, padding=1)



    def forward(self, x):

        h = F.relu(self.conv1_1(x))

        h = F.relu(self.conv1_2(h))

        h = F.max_pool2d(h, 2, 2)



        h = F.relu(self.conv2_1(h))

        h = F.relu(self.conv2_2(h))

        h = F.max_pool2d(h, 2, 2)



        h = F.relu(self.conv3_1(h))

        h = F.relu(self.conv3_2(h))

        h = F.relu(self.conv3_3(h)); f3_3 = h

        h = F.max_pool2d(h, 2, 2)



        h = F.relu(self.conv4_1(h))

        h = F.relu(self.conv4_2(h))

        h = F.relu(self.conv4_3(h)); f4_3 = h

        h = F.max_pool2d(h, 2, 2)



        h = F.relu(self.conv5_1(h))

        h = F.relu(self.conv5_2(h))

        h = F.relu(self.conv5_3(h)); f5_3 = h

        h = F.max_pool2d(h, 2, 2)



        h = F.relu(self.fc6(h))

        h = F.relu(self.fc7(h));     ffc7 = h

        h = F.relu(self.conv6_1(h))

        h = F.relu(self.conv6_2(h)); f6_2 = h

        h = F.relu(self.conv7_1(h))

        h = F.relu(self.conv7_2(h)); f7_2 = h



        f3_3 = self.conv3_3_norm(f3_3)

        f4_3 = self.conv4_3_norm(f4_3)

        f5_3 = self.conv5_3_norm(f5_3)



        cls1 = self.conv3_3_norm_mbox_conf(f3_3)

        reg1 = self.conv3_3_norm_mbox_loc(f3_3)

        cls2 = self.conv4_3_norm_mbox_conf(f4_3)

        reg2 = self.conv4_3_norm_mbox_loc(f4_3)

        cls3 = self.conv5_3_norm_mbox_conf(f5_3)

        reg3 = self.conv5_3_norm_mbox_loc(f5_3)

        cls4 = self.fc7_mbox_conf(ffc7)

        reg4 = self.fc7_mbox_loc(ffc7)

        cls5 = self.conv6_2_mbox_conf(f6_2)

        reg5 = self.conv6_2_mbox_loc(f6_2)

        cls6 = self.conv7_2_mbox_conf(f7_2)

        reg6 = self.conv7_2_mbox_loc(f7_2)



        # max-out background label

        chunk = torch.chunk(cls1,4,1)

        bmax  = torch.max(torch.max(chunk[0],chunk[1]),chunk[2])

        cls1  = torch.cat([bmax,chunk[3]],dim=1)



        return [cls1,reg1,cls2,reg2,cls3,reg3,cls4,reg4,cls5,reg5,cls6,reg6]

Test

In [14]:
from __future__ import print_function



import torch

import torch.nn as nn

import torch.optim as optim

import torch.nn.functional as F

from torch.autograd import Variable

torch.backends.cudnn.bencmark = True



import os,sys,cv2,random,datetime,time,math

import argparse

import numpy as np



import net_s3fd

from bbox import *



def detect(net,img):

    img = img - np.array([104,117,123])

    img = img.transpose(2, 0, 1)

    img = img.reshape((1,)+img.shape)



    img = Variable(torch.from_numpy(img).float(),volatile=True).cuda()

    BB,CC,HH,WW = img.size()

    olist = net(img)



    bboxlist = []

    for i in range(len(olist)/2): olist[i*2] = F.softmax(olist[i*2])

    for i in range(len(olist)/2):

        ocls,oreg = olist[i*2].data.cpu(),olist[i*2+1].data.cpu()

        FB,FC,FH,FW = ocls.size() # feature map size

        stride = 2**(i+2)    # 4,8,16,32,64,128

        anchor = stride*4

        for Findex in range(FH*FW):

            windex,hindex = Findex%FW,Findex//FW

            axc,ayc = stride/2+windex*stride,stride/2+hindex*stride

            score = ocls[0,1,hindex,windex]

            loc = oreg[0,:,hindex,windex].contiguous().view(1,4)

            if score<0.05: continue

            priors = torch.Tensor([[axc/1.0,ayc/1.0,stride*4/1.0,stride*4/1.0]])

            variances = [0.1,0.2]

            box = decode(loc,priors,variances)

            x1,y1,x2,y2 = box[0]*1.0

            # cv2.rectangle(imgshow,(int(x1),int(y1)),(int(x2),int(y2)),(0,0,255),1)

            bboxlist.append([x1,y1,x2,y2,score])

    bboxlist = np.array(bboxlist)

    if 0==len(bboxlist): bboxlist=np.zeros((1, 5))

    return bboxlist



parser = argparse.ArgumentParser(description='PyTorch face detect')

parser.add_argument('--net','-n', default='s3fd', type=str)

parser.add_argument('--model', default='', type=str)

parser.add_argument('--path', default='CAMERA', type=str)



args = parser.parse_args()

use_cuda = torch.cuda.is_available()





net = getattr(net_s3fd,args.net)()

if args.model!='' :net.load_state_dict(torch.load(args.model))

else: print('Please set --model parameter!')

net.cuda()

net.eval()





if args.path=='CAMERA': cap = cv2.VideoCapture(0)

while(True):

    if args.path=='CAMERA': ret, img = cap.read()

    else: img = cv2.imread(args.path)



    imgshow = np.copy(img)

    bboxlist = detect(net,img)



    keep = nms(bboxlist,0.3)

    bboxlist = bboxlist[keep,:]

    for b in bboxlist:

        x1,y1,x2,y2,s = b

        if s<0.5: continue

        cv2.rectangle(imgshow,(int(x1),int(y1)),(int(x2),int(y2)),(0,255,0),1)

    cv2.imshow('test',imgshow)



    if args.path=='CAMERA':

        if cv2.waitKey(1) & 0xFF == ord('q'): break

    else:

        cv2.imwrite(args.path[:-4]+'_output.png',imgshow)

        if cv2.waitKey(0) or True: break

usage: __main__.py [-h] [--net NET] [--model MODEL] [--path PATH]
__main__.py: error: unrecognized arguments: -f C:\Users\hp\AppData\Roaming\jupyter\runtime\kernel-81db5cfc-19fe-49ff-88cd-23cff5aac28c.json


SystemExit: 2

C:\Anaconda3\envs\gpu_env\lib\site-packages\IPython\core\interactiveshell.py:2969: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
import numpy as np

import torch

import torch.nn as nn

import xml.etree.ElementTree as ET



def get_model(path):

    layers = []

    in_channels = 3



    tree = ET.parse(path)

    elems = [elem for elem in tree.iter()]

    elems = elems[::-1]



    for elem in elems:

        meta = elem.attrib

        if elem.tag == 'con' or elem.tag == 'affine_con':

            for key in meta:

                meta[key] = int(meta[key])

            data = ''.join([x for x in elem.text.strip() \

                if x != '\n' and x != '\r'])

            data = [float(x) for x in data.split(' ') if x != '']

            data = np.float32(data)



        if elem.tag == 'con':

            layer = torch.nn.Conv2d(in_channels=in_channels,

                    out_channels=meta['num_filters'],

                    kernel_size = (meta['nr'], meta['nc']),

                    stride = (meta['stride_y'], meta['stride_x']),

                    padding = (meta['padding_y'], meta['padding_x']),

                    bias = True)





            size = meta['num_filters']*meta['nr']*meta['nc']*in_channels



            filter_data = data[ : size]

            bias_data = data[size : ]



            filter_data = np.reshape(filter_data, (meta['num_filters'], \

                in_channels, meta['nr'], meta['nc']))



            layer.weight.data = torch.from_numpy(filter_data)

            layer.bias.data = torch.from_numpy(bias_data)



            layers.append(layer)

            in_channels = meta['num_filters']



            # The dilation can be increased to have a larger receptive field, 

            # for example for use in webcams when the person is 

            # close to the camera.

            layer.dilation = 1



        

        elif elem.tag == 'affine_con':

            layer = torch.nn.BatchNorm2d(num_features=in_channels)



            layer.running_var = torch.from_numpy(np.float32(np.ones(in_channels)))

            layer.running_mean = torch.from_numpy(np.float32(np.zeros(in_channels)))



            layer.weight.data = torch.from_numpy(data[ : in_channels])

            layer.bias.data = torch.from_numpy(data[in_channels : ])



            layers.append(layer)

            

        elif elem.tag == 'relu':

            layers.append(nn.ReLU())



    model = nn.Sequential(*layers)

    return model

In [3]:
import sys

import time

import torch

import torch.nn as nn

import cv2

from torch.autograd import Variable

import numpy as np

from skimage.feature import peak_local_max

from skimage.transform import resize as sk_resize



def recepive_field(model):

	start_in = 0

	j_in = 1

	r_in = 0

	for module in model._modules:

		m = model._modules[module]

		if type(m) is nn.Conv2d:

			if type(m.dilation) is tuple:

				dilation = m.dilation[0]

			else:

				dilation = m.dilation

			kernel_size = m.kernel_size[0] + (m.kernel_size[0]-1)*(dilation-1)

			j_out = j_in * m.stride[0]

			r_out = r_in + (kernel_size - 1) * m.stride[0]

			start_out = start_in + ((kernel_size-1)/2  - m.padding[0])*j_in



			start_in, j_in, r_in = start_out, j_out, r_out

			

	return r_out, start_out



def intersects(rect1, rect2):

    x1, y1, w1, h1, _ = rect1

    x2, y2, w2, h2, _ = rect2

    x_intersection = False

    y_intersection = False

    if x1 <= x2 + w2 and x1 + w1 >= x2:

        x_intersection = True

    if x2 <= x1 + w1 and x2 + w2 >= x1:

        x_intersection = True

    if y1 <= y2 + h2 and y1 + h1 >= y2:

        y_intersection = True

    if y2 <= y1 + h1 and y2 + h2 >= y1:

        y_intersection = True



    if x_intersection and y_intersection:

        return True

    else:

        return False





def detect_img(model, img):

	model.eval()

	orig_shape = img.shape[0 : 2]



	r=122.781998

	g=117.000999

	b=104.297997



	img[: , :, 0] -= b

	img[: , :, 1] -= g

	img[: , :, 2] -= r

	

	img = img[:, :, ::-1].copy() / 256.0



	img = img.transpose((2, 0, 1))

	img = np.float32(img)

	img = torch.from_numpy(img)



	img = img.unsqueeze(0)



	output = model(Variable(img, volatile=True))



	output = output.data.numpy()



	output = output[0, 0, :, :]

	return output



def detect_multi_scale(model, img, scales, recepive_field):

	result = []

	for scale in scales:

			scaled_img = cv2.resize(img, (int(img.shape[1]/scale), int(img.shape[0]/scale)))

			output = detect_img(model, np.float32(scaled_img))

			output = sk_resize(output, output_shape=img.shape[:2], preserve_range=True)

			output = np.float32(output)

			#cv2.imshow("Scale " + str(scale), output)

			#cv2.waitKey(10)

			coordinates = peak_local_max(output, min_distance=30, threshold_abs=-1)

			for y, x in coordinates:

				detection = list(map(int, [x, y, recepive_field*scale, recepive_field*scale, output[y, x]]))

				result.append(detection)



	# Non maximal surpression:

	result = sorted(result, key = lambda r: r[-1], reverse=True)

	after_nms = []

	for detection in result:

		keep_detection = True

		for other_detection in after_nms:

			if intersects(detection, other_detection):

				keep_detection = False

				break

		if keep_detection:

			after_nms.append(detection)



	return after_nms



def detect_webcam(model):

	recepive_field_size, offset = recepive_field(model)

	print ("Receptive field", recepive_field_size, "Center offset", offset)

	scales = [2.7, 3.5]

	img=cv2.imread("wiki_crop/00/440000_1978-07-12_2012.jpg")
    
# 	cap = cv2.VideoCapture(0)


# 	while True:

# 		coordinates = []

# 		ret, img = cap.read()

# 		t0 = time.time()

# 		coordinates = detect_multi_scale(model, img, scales, recepive_field_size)

# 		t1 = time.time()

# 		print("Inference time for frame", t1 - t0)


	coordinates = []

	coordinates = detect_multi_scale(model, img, scales, recepive_field_size)


	for x, y, width, height, score in coordinates:

		x = x - offset//2

		y = y - offset//2

		cv2.rectangle(img, (int(x-width//2, y-height//2)), (int(x+width//2, y+height//2)), (255, 255, 0))

	cv2.imshow("result", img)

	cv2.waitKey()
        
	cv2.destroyAllWindows()


# 		for x, y, width, height, score in coordinates:

# 			x = int(x - offset//2)

# 			y = int(y - offset//2)

# 			cv2.rectangle(img, (x-width//2, y-height//2), (x+width//2, y+height//2), (0, 255, 0))

# 		cv2.imshow("result", img)

# 		cv2.waitKey()
        
# 		cv2.destroyAllWindows()


if __name__ == '__main__':
    
	model = get_model("face.xml")

	detect_webcam(model)

Receptive field 44 Center offset 14.0


C:\Anaconda3\envs\gpu_env\lib\site-packages\ipykernel\__main__.py:141: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
C:\Anaconda3\envs\gpu_env\lib\site-packages\skimage\transform\_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Anaconda3\envs\gpu_env\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
